In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# 사용할 디바이스 설정 (GPU가 있다면 GPU 사용)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# # 랜덤 시드 고정 (재현성을 위해, 주석 처리됨)
# torch.manual_seed(0)  # PyTorch의 난수 생성기 시드 고정
# if device == 'cuda':
#     torch.cuda.manual_seed_all(0)  # CUDA를 사용하는 경우 시드 고정

# 데이터 변환 정의
# MNIST 이미지는 기본적으로 [0, 255] 범위의 픽셀 값을 가짐.
# ToTensor()를 사용하면 [0, 1] 범위로 정규화됨.
# 이후 Normalize((0.5,), (0.5,))를 적용하면 [-1, 1] 범위로 정규화됨.
transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 PyTorch Tensor 형식으로 변환
    transforms.Normalize((0.5,), (0.5,))  # 평균 0.5, 표준편차 0.5로 정규화
])

train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# 하이퍼파라미터 설정
batch_size = 64  # 한 번에 처리할 데이터 샘플 개수 (미니배치 크기)
learning_rate = 0.001  # 학습률 (신경망 가중치 업데이트 비율)
num_epochs = 10  # 전체 데이터셋을 학습하는 횟수 (에포크 수)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,  # 사용할 데이터셋
    batch_size=batch_size,  # 미니배치 크기 설정
    shuffle=True,  # 데이터를 섞어서 학습하도록 설정 (일반적으로 학습 데이터에는 shuffle=True 사용)
    drop_last=True  # 마지막 배치 크기가 batch_size보다 작으면 버림 (연산 안정성을 위해)
)

# test_loader: 테스트 데이터셋을 로드하는 데이터로더
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,  # 사용할 데이터셋
    batch_size=batch_size,  # 미니배치 크기 설정
    shuffle=False  # 테스트 데이터는 순서를 유지해야 하므로 shuffle=False 사용
)

Using device: cuda


100%|██████████| 9.91M/9.91M [00:00<00:00, 17.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 458kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.36MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.07MB/s]


In [ ]:
import torch

# CNN 모델 정의 (PyTorch의 nn.Module을 상속받아 구현)
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()  # 부모 클래스 초기화

        # 첫 번째 합성곱 계층 (Convolutional Layer 1)
        # 입력 채널: 1 (MNIST는 흑백 이미지이므로 입력 채널 1)
        # 출력 채널: 32 (필터 32개 사용)
        # 커널 크기: 3x3 (3x3 크기의 필터 적용)
        # 패딩: 1 (출력 크기를 유지하기 위해 padding 추가)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),  # 합성곱 연산
            torch.nn.BatchNorm2d(32),  # 배치 정규화 (학습 안정화 및 속도 향상)
            torch.nn.ReLU(),  # 활성화 함수 (비선형성 추가)
            torch.nn.MaxPool2d(kernel_size=2, stride=2)  # 2x2 최대 풀링 (특성 맵 크기 절반 감소)
        )
        # 입력 크기: (1, 28, 28) → 출력 크기: (32, 14, 14)

        # 두 번째 합성곱 계층 (Convolutional Layer 2)
        # 입력 채널: 32 (이전 계층의 출력이 입력이 됨)
        # 출력 채널: 64 (필터 64개 사용)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),  # 합성곱 연산
            torch.nn.BatchNorm2d(64),  # 배치 정규화
            torch.nn.ReLU(),  # 활성화 함수
            torch.nn.MaxPool2d(kernel_size=2, stride=2)  # 2x2 최대 풀링
        )
        # 입력 크기: (32, 14, 14) → 출력 크기: (64, 7, 7)

        # 완전 연결층 (Fully Connected Layer)
        # 합성곱 계층을 통과한 후, 특성 맵 크기는 (64, 7, 7)이 됨.
        # 이를 FC 레이어에 입력하기 위해 Flatten하여 3136(=64×7×7) 차원 벡터로 변환
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(7 * 7 * 64, 128),  # 완전 연결층 1 (입력: 3136, 출력: 128)
            torch.nn.ReLU(),  # 활성화 함수
            torch.nn.Dropout(0.5),  # 과적합 방지를 위한 드롭아웃 (50% 확률로 뉴런 비활성화)
            torch.nn.Linear(128, 10)  # 완전 연결층 2 (출력: 10개 클래스, MNIST 숫자 0~9)
        )

        # 가중치 초기화 (Xavier 초기화 적용)
        torch.nn.init.xavier_uniform_(self.fc[0].weight)

    # 순전파(Forward) 연산 정의
    def forward(self, x):
        x = self.layer1(x)  # 첫 번째 합성곱 계층 통과
        x = self.layer2(x)  # 두 번째 합성곱 계층 통과
        x = x.view(x.size(0), -1)  # Flatten (완전 연결층 입력을 위해 1차원 벡터로 변환)
        x = self.fc(x)  # 완전 연결층 통과하여 최종 출력값 계산
        return x  # 모델의 출력 반환 (10개의 클래스에 대한 예측값)

In [ ]:
# CNN 모델 인스턴스 생성 후 지정한 장치(GPU or CPU)로 이동
model = CNN().to(device)

# 손실 함수 정의
# CrossEntropyLoss: 분류 문제에서 많이 사용하는 손실 함수
# label_smoothing=0.1: 정답 레이블을 100% 신뢰하지 않고 10% 정도 분산을 둬서 학습 안정성을 높임
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1).to(device)

# 옵티마이저(Adam) 정의
# model.parameters(): 모델의 학습 가능한 파라미터들
# lr=0.001: 학습률(learning rate), 가중치 업데이트의 크기를 조절
# weight_decay=1e-4: 가중치 감소 (L2 정규화) → 과적합 방지
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# 학습률 스케줄러 정의 (StepLR)
# StepLR: 일정한 에포크(step_size)마다 학습률을 gamma 비율만큼 감소시킴
# step_size=5: 5 에포크마다 학습률 감소
# gamma=0.5: 학습률을 50%로 감소 (예: 0.001 → 0.0005 → 0.00025 ...)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [ ]:
# 학습 및 평가 루프
best_accuracy = 0  # 최고 테스트 정확도를 저장할 변수

# 전체 학습 횟수(epoch)만큼 반복
for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정 (Dropout, BatchNorm 등이 학습 모드로 동작)
    avg_cost = 0  # 한 epoch 동안의 평균 손실 초기화

    # 훈련 데이터셋을 한 배치씩 가져와 학습
    for X, Y in train_loader:
        X, Y = X.to(device), Y.to(device)  # 데이터를 GPU 또는 CPU로 이동

        optimizer.zero_grad()  # 기울기(Gradient) 초기화 (이전 업데이트 정보 제거)
        hypothesis = model(X)  # 모델에 입력 데이터(X)를 전달하여 예측 수행
        cost = criterion(hypothesis, Y)  # 예측값(hypothesis)과 실제 정답(Y) 비교하여 손실 계산
        cost.backward()  # 역전파(Backpropagation) 수행 (기울기 계산)
        optimizer.step()  # 옵티마이저를 사용하여 가중치 업데이트
        avg_cost += cost / len(train_loader)  # 배치별 손실을 누적하여 평균 손실 계산

    # 학습률 업데이트 (StepLR 사용)
    scheduler.step()

    # 테스트 정확도 평가 (모델 검증)
    model.eval()  # 모델을 평가 모드로 설정 (Dropout, BatchNorm 비활성화)
    test_accuracy = 0  # 테스트 정확도 저장 변수 초기화

    with torch.no_grad():  # 평가 과정에서는 기울기 계산 안 함 (메모리 절약 및 속도 향상)
        for X_test, Y_test in test_loader:
            X_test, Y_test = X_test.to(device), Y_test.to(device)  # 데이터를 GPU 또는 CPU로 이동
            prediction = model(X_test)  # 모델에 입력하여 예측 수행
            test_accuracy += (torch.argmax(prediction, 1) == Y_test).float().mean()
            # torch.argmax(prediction, 1): 가장 확률이 높은 클래스 예측
            # == Y_test: 정답과 비교하여 맞춘 개수 계산
            # .mean(): 배치 단위 정확도 계산 후 누적

    test_accuracy /= len(test_loader)  # 전체 배치 수로 나눠서 평균 정확도 계산

    # 최고 정확도 업데이트
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy  # 새로운 최고 정확도로 갱신

    # 학습 결과 출력
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_cost:.4f}, Test Accuracy: {test_accuracy*100:.2f}%")

# 최종 최고 테스트 정확도 출력
print(f"Best Test Accuracy: {best_accuracy*100:.2f}%")

Epoch [1/10], Loss: 0.7969, Test Accuracy: 98.42%
Epoch [2/10], Loss: 0.6849, Test Accuracy: 98.87%
Epoch [3/10], Loss: 0.6643, Test Accuracy: 98.94%
Epoch [4/10], Loss: 0.6516, Test Accuracy: 99.10%
Epoch [5/10], Loss: 0.6409, Test Accuracy: 99.19%
Epoch [6/10], Loss: 0.6212, Test Accuracy: 99.32%
Epoch [7/10], Loss: 0.6132, Test Accuracy: 99.31%
Epoch [8/10], Loss: 0.6072, Test Accuracy: 99.26%
Epoch [9/10], Loss: 0.5993, Test Accuracy: 99.26%
Epoch [10/10], Loss: 0.5926, Test Accuracy: 99.28%
Best Test Accuracy: 99.32%
